In [15]:
import kfp.dsl as dsl
import kfp.components as comp
from kubernetes import client as k8s_client


def init() -> str:
    import os
    import numpy as np
    _kale_data_directory = "/data/_kale_data/"
    a = np.random.random((10, 10))
    if not os.path.isdir(_kale_data_directory):
        os.makedirs(_kale_data_directory, exist_ok=True)
    np.save(_kale_data_directory + 'a.npy', a)

def sum(arg0) -> str: 
    import os
    import numpy as np
    _kale_data_directory = "/data/_kale_data/"
    a = np.load(_kale_data_directory + "a.npy")
    b = a.sum()
    print(b)

def mean(arg0) -> str:
    import os
    import numpy as np
    _kale_data_directory = "/data/_kale_data/"
    a = np.load(_kale_data_directory + "a.npy")
    c = a.mean()
    print(c)


init_op = comp.func_to_container_op(
    init, base_image='stefanofioravanzo/kale-kfp-examples:0.1')
sum_op = comp.func_to_container_op(
    sum, base_image='stefanofioravanzo/kale-kfp-examples:0.1')
mean_op = comp.func_to_container_op(
    mean, base_image='stefanofioravanzo/kale-kfp-examples:0.1')

_KALE_PVC_NAME = "kale_pvc"
@dsl.pipeline(
    name='test_pipeline_numpy',
    description='Test Pipeline from CLI'
)
def test_pipeline_numpy():
    
    vop = dsl.VolumeOp(
        name=_KALE_PVC_NAME,
        resource_name="kale-marshal-pvc",
        modes=dsl.VOLUME_MODE_RWM,
        size="512Mi"
    )
    
    pvc = k8s_client.V1PersistentVolumeClaim(...)
    vop = dsl.VolumeOp(
        name="pvc-creation",
        k8s_resource=pvc
    )

    init_task = init_op()\
        .add_pvolumes({'/data': vop.volume})

    sum_task = sum_op(init_task.output).add_pvolumes({'/data': init_task.pvolume})

    mean_task = mean_op(init_task.output).add_pvolumes({'/data': init_task.pvolume})
    
    
    
    vol_dict = {
        "apiVersion": "v1",
        "kind": "PersistentVolumeClaim",
        "metadata": {
            "name": vop.outputs['name']
        }
    }

    delete_vol = dsl.ResourceOp(
        name="kale-pvc-volume-deletion",
        action="delete",
        k8s_resource=vol_dict
    ).after(sum_task, mean_task)



pipeline_func = test_pipeline_numpy
pipeline_filename = pipeline_func.__name__ + '.pipeline.tar.gz'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)